In [156]:
import pandas as pd
import unicodedata
from nltk.corpus import stopwords
import nltk
import re
import math
import spotlight
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
from io import StringIO
import os
import spacy
from spacy import displacy
from collections import Counter
from pprint import pprint
import en_core_web_sm
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix
from sklearn.metrics import f1_score
from dateparser.search import search_dates
import datetime

nltk.download("stopwords")
stop_words = stopwords.words('english')

java_path = "C:\\Program Files\\Java\\jdk1.8.0_231\\bin\\java.exe"
os.environ['JAVAHOME'] = java_path

nlp = en_core_web_sm.load()
nltk.download('wordnet')
nltk.download("stopwords")
lemmatizer = WordNetLemmatizer()
pd.set_option('display.max_colwidth', -1)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kadss\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kadss\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kadss\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# file load, cleaning and unicode conversion

file_name='Boston2C.csv'

for_pd = StringIO()
with open('Data/2CVTweets/'+file_name,encoding="utf8") as f:
    for line in f:
        new_line = re.sub(r'NO;', 'NO',line)  ##cleaning 'NO'
        print (new_line, file=for_pd)

for_pd.seek(0)

df_city = pd.read_csv(for_pd, sep=';', header=None)
df_city = df_city.drop(df_city.columns[0], axis=1) ## index column
df_city.columns=['tweet','Y']


In [136]:
df_city.head(20)

,tweet,Y,tweet_org,tweet_tok_lem,no_ques_marks,no_excl_marks,no_uppercase,annotations,num_same_URI,acc_overlap_types,non_acc_overlap_types,num_locations
0,"b'RT' b'@CommuterBoston:' b'ACCIDENT' b'(Wellesley,' b'MA):' b'RT-9' b'East' b'near' b'Cedar' b'St' b'-' b'involving' b'a' b'motorcycle' b'and' b'a' b'car' b'-' b'possible' b'lane' b'restrictions'",YES,"RT @CommuterBoston: ACCIDENT (Wellesley, MA): RT-9 East near Cedar St - involving a motorcycle and a car - possible lane restrictions","[b'RT, ', b, ', @, CommuterBoston, :, ', b'ACCIDENT, ', b, ', (, Wellesley, ,, ', b'MA, ), :, ', b'RT-9, ', b'East, ', b'near, ', b'Cedar, ', b'St, ', b'-, ', b'involving, ', b, ', a, ', b'motorcycle, ', b'and, ', b, ', a, ', b'car, ', b'-, ', b'possible, ', b'lane, ', b'restrictions, ']",0,0,0,"{'URI': ['http://dbpedia.org/resource/Sacramento_Regional_Transit_District', 'http://dbpedia.org/resource/Master_of_Arts', 'http://dbpedia.org/resource/Sacramento_Regional_Transit_District', 'http://dbpedia.org/resource/Motorcycle'], 'types': ['Wikidata:Q43229,Wikidata:Q24229398,DUL:SocialPerson,DUL:Agent,Schema:Organization,DBpedia:Organisation,DBpedia:Company,DBpedia:BusCompany,DBpedia:Agent', '', 'Wikidata:Q43229,Wikidata:Q24229398,DUL:SocialPerson,DUL:Agent,Schema:Organization,DBpedia:Organisation,DBpedia:Company,DBpedia:BusCompany,DBpedia:Agent', '']}",2,0.023810,0.014493,1
1,b'RT' b'@CommuterBoston:' b'ACCIDENT:' b'I-290' b'East' b'before' b'I-495' b'(Exit' b'#26)' b'-' b'2-car' b'accident' b'with' b'possible' b'lane' b'blockages',YES,RT @CommuterBoston: ACCIDENT: I-290 East before I-495 (Exit #26) - 2-car accident with possible lane blockages,"[b'RT, ', b, ', @, CommuterBoston, :, ', b'ACCIDENT, :, ', b, ', I-290, ', b'East, ', b'before, ', b, ', I-495, ', b, ', (, Exit, ', b, ', #, 26, ), ', b'-, ', b, ', 2-car, ', b'accident, ', b'with, ', b'possible, ', b'lane, ', b'blockages, ']",0,0,0,"{'URI': ['http://dbpedia.org/resource/Sacramento_Regional_Transit_District', 'http://dbpedia.org/resource/Interstate_290_(Illinois)', 'http://dbpedia.org/resource/Falmouth_Spur'], 'types': ['Wikidata:Q43229,Wikidata:Q24229398,DUL:SocialPerson,DUL:Agent,Schema:Organization,DBpedia:Organisation,DBpedia:Company,DBpedia:BusCompany,DBpedia:Agent', 'Wikidata:Q34442,Schema:Place,DBpedia:RouteOfTransportation,DBpedia:Road,DBpedia:Place,DBpedia:Location,DBpedia:Infrastructure,DBpedia:ArchitecturalStructure', 'Wikidata:Q34442,Schema:Place,DBpedia:RouteOfTransportation,DBpedia:Road,DBpedia:Place,DBpedia:Location,DBpedia:Infrastructure,DBpedia:ArchitecturalStructure']}",0,0.023810,0.014493,0
2,b'Opened' b'report' b'via' b'iPhone' b'at' D b'Standard' b'St' b'URL,NO,Opened report via iPhone at 19 Standard St http://t.co/8vc761aCHT. Road kill on sidewalk.,"[b'Opened, ', b'report, ', b'via, ', b'iPhone, ', b'at, ', D, b'Standard, ', b'St, ', b'URL]",0,0,1,"{'URI': ['http://dbpedia.org/resource/IPhone', 'http://dbpedia.org/resource/Twitter', 'http://dbpedia.org/resource/Roadkill', 'http://dbpedia.org/resource/Sidewalk'], 'types': ['Wikidata:Q1067263,DBpedia:InformationAppliance,DBpedia:Device', 'Wikidata:Q43229,Wikidata:Q24229398,DUL:SocialPerson,DUL:Agent,Schema:Organization,DBpedia:Organisation,DBpedia:Company,DBpedia:Agent', '', '']}",0,0.035714,0.021739,0
3,b'RT' b'@CommuterBoston:' b'ACCIDENT' b'REPORTED:' b'I-93' b'North' b'before' b'Granite' b'Av' b'(Exit' b'#11)' b'-' b'police' b'investigating',YES,RT @CommuterBoston: ACCIDENT REPORTED: I-93 North before Granite Av (Exit #11) - police investigating,"[b'RT, ', b, ', @, CommuterBoston, :, ', b'ACCIDENT, ', b'REPORTED, :, ', b, ', I-93, ', b'North, ', b'before, ', b'Granite, ', b'Av, ', b, ', (, Exit, ', b, ', #, 11, ), ', b'-, ', b'police, ', b'investigating, ']",0,0,0,"{'URI': ['http://dbpedia.org/resource/Sacramento_Regional_Transit_District', 'http://dbpedia.org/resource/Interstate_93'], 'types': ['Wikidata:Q43229,Wikidata:Q24229398,DUL:SocialPerson,DUL:Agent,Schema:Organization,DBpedia:Organisation,DBpedia:Company

In [21]:
#unicode conversion
df_city['tweet_org']=df_city['tweet']
df_city['tweet'] = (df_city['tweet'].map(lambda x: unicodedata.normalize('NFKD', x))
              .str.encode('ascii', 'ignore'))

In [ ]:
# slang conversion

In [22]:
slang_covert_url='http://www.netlingo.com/acronyms.php'
slangdict= {}


from bs4 import BeautifulSoup
import requests, json
resp = requests.get('http://www.netlingo.com/acronyms.php')
soup = BeautifulSoup(resp.text, "html.parser")

key=""
value=""
for div in soup.findAll('div', attrs={'class':'list_box3'}):
    
    for li in div.findAll('li'):
        for a in li.findAll('a'):
            key =a.text
            value = li.text.split(key)[1]
            slangdict[key.lower()]=value
            

        

In [23]:
def slang_to_formal(input):
    i=1
    formal=[]
    for slang in input.split():
        i=i+1
        if slang.lower() in slangdict:
            formal.append(slangdict[slang])
        elif slang.isdigit():
            
            formal.append('D')  #not working  (replacing digits)
        else:
            formal.append(slang)
       
        
    return  " ".join(str(x) for x in formal)
            

In [24]:
df_city['tweet'] = df_city.apply(lambda row: slang_to_formal(row['tweet']), axis=1)

In [25]:
#replacing URL and digits

In [26]:
def replace_URL(tweet):
    return re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 'URL', tweet)

df_city['tweet'] = df_city.apply(lambda row: replace_URL(row['tweet']), axis=1)

In [28]:
#remove stop words

stop = stopwords.words('english')
df_city['tweet'] = df_city.apply(lambda x: [item for item in x if item not in stop])

In [29]:
#tokenisation

df_city['tweet_tok_lem'] = df_city.apply(lambda row: nltk.word_tokenize(row['tweet']), axis=1)

In [30]:
#lemmatize
def lemmatize(s):
       
    s = [lemmatizer.lemmatize(word) for word in s]
    return s

df_city['tweet_tok_lem'] = df_city.apply(lambda row: lemmatize(row['tweet_tok_lem']), axis=1)


In [31]:
#syntactic features

In [33]:
df_city['no_ques_marks']=df_city.apply(lambda row: (row['tweet'].count("?")), axis=1)
df_city['no_excl_marks']=df_city.apply(lambda row: (row['tweet'].count("!")), axis=1)
df_city['no_uppercase']=df_city.apply(lambda row: sum(map(str.isupper, row['tweet'].split())), axis=1)

In [34]:

#Number same URI (NER-DBPedia)

In [35]:
from collections import Counter

def duplicates(values):
    
    dups = Counter(values) - Counter(set(values))
    return sum(dups.values()) + len(dups)

In [36]:
#DBPedia api hit
def get_annotation(tweet):
    try:
        annotations = spotlight.annotate('http://api.dbpedia-spotlight.org/en/annotate',tweet,
                              confidence=0.4, support=20)
    except Exception:
        return {'URI':[],'types':[]}
    
    URI_lst=[]
    types_lst=[]
    
    for ann_dict in annotations:
        URI_lst.append(ann_dict['URI'])
        types_lst.append(ann_dict['types'])
    
    return {'URI':URI_lst,'types':types_lst}

In [39]:
#annotation features

#global 
accident_types=set()
non_accident_types=set()
    
def create_accident_types(ann_types,Y):
       
    for types in ann_types:
        if Y=='YES':
            accident_types.add(types)
        else:
            non_accident_types.add(types)
 


In [40]:
df_city['annotations'] = df_city.apply(lambda row: get_annotation(row['tweet_org']), axis=1)



In [41]:
# creating accident and non-accident frequency distribution
for ann,Y in zip(df_city.annotations, df_city.Y):
    if 'types' in ann:
        create_accident_types(ann['types'],Y)
                              

In [42]:
# NER features

df_city['num_same_URI'] = df_city.apply(lambda row: duplicates(row['annotations']['URI']), axis=1)
df_city['acc_overlap_types'] = df_city.apply(lambda row: (len(accident_types.intersection(row['annotations']['types']))/len(accident_types)), axis=1)
df_city['non_acc_overlap_types'] = df_city.apply(lambda row: (len(non_accident_types.intersection(row['annotations']['types']))/len(non_accident_types)), axis=1)

In [176]:
def get_location_count(text):
    doc = nlp(text)
    return sum([1  for X in doc.ents if X.label_=='GPE'])


def replace_LOC(text):
    doc = nlp(text)
    result_tweet=''
    abs_dict={X.text:'LOC' for X in doc.ents if X.label_=='GPE'}
   # loc_dict={n:'LOC' for n in range(1,10) if n==1}
    dates = search_dates(text)
    if dates is not None:
      
        for tok in dates:

            if isinstance(tok[1], datetime.datetime):
              
                text=text.replace(tok[0],'DATETIME')

        
    for word in text.split():
        if word in abs_dict:
            word=abs_dict[word]
        
        result_tweet=result_tweet+' '+word
        
        
    return(result_tweet.strip())
    
    
    


replace_LOC('I am in USA Dallas at 10 am')

    

'I am in LOC LOC DATETIME'

In [ ]:
df_city['num_locations'] = df_city.apply(lambda row: get_location_count(row['tweet']), axis=1)
df_city['replaced_twet'] = df_city.apply(lambda row: replace_LOC(row['tweet']), axis=1)

## Modeling

In [52]:
from sklearn.model_selection import train_test_split

X =df_city[['tweet','no_ques_marks', 'no_excl_marks', 'no_uppercase', 'annotations',
       'num_same_URI', 'acc_overlap_types', 'non_acc_overlap_types',
       'num_locations']]
Y = df_city.loc[:, 'Y']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)


In [53]:
# create tweets corpus
all_tweets_corpus=df_city['tweet'].tolist()

In [ ]:
#Pipeline Preprocessing


In [54]:
#classes to return text and numeric values for transformer

from sklearn.base import BaseEstimator, TransformerMixin
class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.field]
class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[[self.field]]
    

In [55]:
def lemmatize(s):
    s = [lemmatizer.lemmatize(word) for word in s]
    return s
    
def Tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    return lemmatize(words)

In [56]:
#Pipeline

In [57]:
classifier = Pipeline([
    ('features', FeatureUnion([
        ('tfidf_reduced', Pipeline([
            ('colext', TextSelector('replaced_twet')),
            ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stop_words,
                     min_df=.0025, max_df=0.25, ngram_range=(1,2))),
            ('svd', TruncatedSVD(algorithm='randomized', n_components=300)), 
        ])),
        ('question_marks', Pipeline([
            ('wordext', NumberSelector('no_ques_marks')),
            ('wscaler', StandardScaler()),
        ])),
         ('no_excl_marks', Pipeline([
            ('wordext', NumberSelector('no_excl_marks')),
            ('wscaler', StandardScaler()),
        ])),
         ('no_uppercase', Pipeline([
            ('wordext', NumberSelector('no_uppercase')),
            ('wscaler', StandardScaler()),
        ])),
         ('num_locations', Pipeline([
            ('wordext', NumberSelector('num_locations')),
            ('wscaler', StandardScaler()),
        ])),
         ('num_same_URI', Pipeline([
            ('wordext', NumberSelector('num_same_URI')),
            ('wscaler', StandardScaler()),
        ])),
         ('acc_overlap_types', Pipeline([
            ('wordext', NumberSelector('acc_overlap_types')),
        
        ])),
         ('non_acc_overlap_types', Pipeline([
            ('wordext', NumberSelector('non_acc_overlap_types')),
           
        ])),
    ])),
#    ('clf', XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.1)),
    ('rf', RandomForestClassifier()),
    ])

In [58]:
# fitting & prediction

classifier.fit(X_train, y_train)
preds = classifier.predict(X_test)


C:\Users\kadss\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['doe', 'ha', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
C:\Users\kadss\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [60]:
# Evaluation

print("Accuracy:", accuracy_score(y_test, preds))
print(classification_report(y_test, preds))
print(confusion_matrix(y_test, preds))
print("F1-Score:",f1_score(y_test, preds, average='weighted')) 

Accuracy: 0.8822695035460993
              precision    recall  f1-score   support

          NO       0.88      0.99      0.93       571
         YES       0.88      0.44      0.59       134

    accuracy                           0.88       705
   macro avg       0.88      0.71      0.76       705
weighted avg       0.88      0.88      0.87       705

[[563   8]
 [ 75  59]]
F1-Score: 0.8659099308378649
